# **REPRESENTATION 1**

In [2]:
import numpy as np

def init_board(n):
    # edges[i,j,0] = horizontal edge between (i,j) and (i,j+1)
    # edges[i,j,1] = vertical edge between (i,j) and (i+1,j)
    edges = np.zeros((n+1, n+1, 2), dtype=int)
    boxes = np.zeros((n, n), dtype=int)  # track what boxes are completed
    return edges, boxes


def boxes_from_edge(i, j, k, n, edges):
    """Return list of (r, c) boxes completed by placing edge (i,j,k)."""
    completed = []

    if k == 0:  # horizontal edge
        # Box below this edge (if not last row)
        if i < n:
            if (edges[i, j, 0] and edges[i, j+1, 1] and
                edges[i, j, 1] and edges[i+1, j, 0]):
                completed.append((i, j))
        # Box above this edge (if not first row)
        if i > 0:
            if (edges[i-1, j, 0] and edges[i-1, j, 1] and
                edges[i-1, j+1, 1] and edges[i, j, 0]):
                completed.append((i-1, j))

    elif k == 1:  # vertical edge
        # Box to the right (if not last column)
        if j < n:
            if (edges[i, j, 1] and edges[i, j, 0] and
                edges[i+1, j, 0] and edges[i, j+1, 1]):
                completed.append((i, j))
        # Box to the left (if not first column)
        if j > 0:
            if (edges[i, j-1, 1] and edges[i, j-1, 0] and
                edges[i+1, j-1, 0] and edges[i, j, 1]):
                completed.append((i, j-1))

    return completed


# ---------------- Test ----------------
n = 2
edges, boxes = init_board(n)

# Draw 3 edges of box (0,0)
edges[0,0,0] = 1  # top
edges[0,0,1] = 1  # left
edges[1,0,0] = 1  # bottom (this one should complete the box)
edges[0,1,0] = 1
edges[0,2,1] = 1
edges[1,1,0] = 1
edges[0,1,1] = 1  # right

# completed = boxes_from_edge(1,0,0,n,edges)
# print("Completed boxes:", completed)

completed = boxes_from_edge(0,1,1,n,edges)
print("Completed boxes:", completed)

Completed boxes: [(0, 1), (0, 0)]


# **REPRESENTATION 2**

In [11]:
import numpy as np

In [18]:
# game.py

# Going to change to a class instead for ease?
class dots_and_boxes():
  def __init__(self, n):
    self.n = n
    """
    Initialize the 5 layers that represent a game state
    board_tensor: shape (n, n, 5) with 4 edges per box and an extra channel for player ownership
                  channels are in [top, bottom, left, right, player owner]
    boxes_playerA: list of boxes owned by player A in (x,y)
    boxes_playerB: list of boxes owned by player B
    recent_edge: most recent edge drawn
    On_Offensive: boolean variable representing whether a player has just completed a box and is going again
    """
    self.board_tensor = np.zeros((n, n, 5), dtype=int)
    self.boxes_playerA = []
    self.boxes_playerB = []
    self.recent_edge = None
    self.On_Offensive = False

  def set_edge(self, r, c, edge_type):
    """
    Set the edge of box (r,c)
    edge_type: "top", "bottom", "left", "right"
    Automatically updates neighboring box if shared
    Updates recent_edge
    """
    n = self.n
    self.recent_edge = [r,c,edge_type]

    if edge_type == "top":
        self.board_tensor[r, c, 0] = 1
        if r > 0:
            self.board_tensor[r-1, c, 1] = 1  # bottom of box above
    elif edge_type == "bottom":
        self.board_tensor[r, c, 1] = 1
        if r < n-1:
            self.board_tensor[r+1, c, 0] = 1  # top of box below
    elif edge_type == "left":
        self.board_tensor[r, c, 2] = 1
        if c > 0:
          self.board_tensor[r, c-1, 3] = 1  # right of box to the left
    elif edge_type == "right":
        self.board_tensor[r, c, 3] = 1
        if c < n-1:
          self.board_tensor[r, c+1, 2] = 1  # left of box to the right

  def check_box(self, r, c, edge_type, player):
    """
    Checks if all edges of box (r,c) are drawn
    If a box is created, assigns ownership
    Also checks if additonal box next to (r,c) is completed
    edge_type: "top", "bottom", "left", "right"
    player: 1,2
    Automatically updates boxes_playerA and boxes_playerB
    Automatically updates On_Offensive
    """
    n = self.n
    current_box = False

    if np.all(self.board_tensor[r, c, :4] == 1):
      current_box = True

      # Updating boxes_playerA or boxes_playerB
      self.board_tensor[r,c,4] = player # assigns ownership
      if player == 1:
        self.boxes_playerA.append((r,c))
      else:
        self.boxes_playerB.append((r,c))

    additional_box = False

    # Used to update the boxes under each player with the appropriate additional box
    additional_r = r
    additional_c = c

    if edge_type == "top":
      if r > 0:  # check box above
        if np.all(self.board_tensor[r-1, c, :4] == 1): # all edges are filled in
          additional_box = True
          self.board_tensor[r-1, c, 4] = player # assigns ownership
          additional_r = r-1

    elif edge_type == "bottom":
      if r < (n-1):  # check box below
        if np.all(self.board_tensor[r+1, c, :4] == 1): # all edges are filled in
          additional_box = True
          self.board_tensor [r+1, c, 4] = player # assigns ownership
          additional_r = r+1

    elif edge_type == "left":
      if c > 0: # check the box to the left
        if np.all(self.board_tensor[r,c-1, :4] == 1):
          additional_box = True
          self.board_tensor[r, c-1, 4] = player # assigns ownership
          additional_c = c-1

    elif edge_type == "right":
      if c < (n-1): # check the box to the right
        if np.all(self.board_tensor[r, c+1, :4] == 1):
          additional_box = True
          self.board_tensor [r, c+1, 4] = player # assigns ownership
          additional_c = c+1

    # Updating boxes_playerA or boxes_playerB
    if additional_box:
      if player == 1:
        self.boxes_playerA.append((additional_r,additional_c))
      else:
        self.boxes_playerB.append((additional_r,additional_c))

    # Updating On_Offensive
    self.On_Offensive = current_box or additional_box

    return int(current_box) + int(additional_box)


  def print_board_ascii(self, move, player, initial = False):
    """
    Print the current board
    move: move number
    player: player number
    initial: True if first time showing the board
    """
    n = self.n
    board_tensor = self.board_tensor
    BOX_WIDTH = 7

    #ansi color codes
    RED = "\033[1;91m"
    BLUE = "\033[1;94m"
    END = "\033[0m"


    if(initial):
      print()
      intro = "Welcome!"
      intro = intro.center(BOX_WIDTH * n + 5)
      print(intro)
      welcome = "Here is the board"
      welcome = welcome.center(BOX_WIDTH * n + 5)
      print(welcome)
    else:
      player_intro = f"Player {player} , Move {move}"
      player_intro = player_intro.center(BOX_WIDTH * n + 5)
      print(player_intro)
    for r in range(n):
        # Print top edges
        top_row = " "
        for c in range(n):
            top_row += "+" #prints all points
            top_row += "-" * BOX_WIDTH if board_tensor[r, c, 0] else " " * BOX_WIDTH # prints a horizontal line the same length as box width
        top_row += "+"
        print(top_row)

        # Print vertical edges and ownership/coordinates
        mid_row = " "
        for c in range(n):
            mid_row += "|" if board_tensor[r, c, 2] else " "

            # show coordinates unless box is owned (shows owner in that case)
            owner = board_tensor[r, c, 4]

            #if box made, add color+ownership display
            if owner == 1:
               content = BLUE + "P1".center(BOX_WIDTH) + END #player 1 marking
            elif owner == 2:
               content = RED + "P2".center(BOX_WIDTH) + END #player 2 marking
            else:
               content = f"({r},{c})".center(BOX_WIDTH)  #box not completed by either yet

            mid_row += content

        # last right edge of the row
        mid_row += "|" if board_tensor[r, n-1, 3] else " "
        print(mid_row)

    # Print bottom edges of last row
    bottom_row = " "
    for c in range(n):
        bottom_row += "+"
        bottom_row += "-" * BOX_WIDTH if board_tensor[n-1, c, 1] else " " * BOX_WIDTH # prints a horizontal line the same length as box width
    bottom_row += "+"
    print(bottom_row)
    print()

  def player_turn(self, r, c, edge_type, player,  move):
  # Essentially just instantiates other functions that are used in a turn
    self.set_edge(r, c, edge_type)
    completed_box = self.check_box(r, c, edge_type, player) # if want to find out how many boxes made by a move
    self.print_board_ascii(move, player)
    return completed_box



In [8]:
# @title

# OLD CODE TESTING INITIALIZATION AND SETTING EDGE AND CHECKING IF BOXES COMPLETE AND DRAWING THEM
# Initialize 2x2 board
# board = init_box_tensor(2)

# completed_boxes_player1 = 0
# completed_boxes_player2 = 0

# # Draw the top edge of box (0,0)
# set_edge(board, 0, 0, "top")
# completed_boxes_player1 += check_box(board, 0, 0, "top", 1)
# print_board_ascii(board, 1, 1)

# # Draw the left edge of box (0,0)
# set_edge(board, 0, 0, "left")
# completed_boxes_player2 += check_box(board, 0, 0, "left", 2)
# print_board_ascii(board, 2, 2)

# # Draw the bottom edge of box (0,0)
# set_edge(board, 0, 0, "bottom")
# completed_boxes_player1 += check_box(board, 0, 0, "bottom", 1)
# print_board_ascii(board, 3, 1)

# # Draw the top edge of box (0,1)
# set_edge(board, 0, 1, "top")
# completed_boxes_player2 += check_box(board, 0, 1, "top", 2)
# print_board_ascii(board, 4, 2)

# # Draw the right edge of box (0,1)
# set_edge(board, 0, 1, "right")
# completed_boxes_player1 += check_box(board, 0, 1, "right", 1)
# print_board_ascii(board, 5, 1)

# # Draw the bottom edge of box (0,1)
# set_edge(board, 0, 1, "bottom")
# completed_boxes_player2 += check_box(board, 0, 1, "bottom", 2)
# print_board_ascii(board, 6, 2)

# # Draw the left edge of box (0,1) => completes 2 boxes for player 1
# set_edge(board, 0, 1, "left")
# completed_boxes_player1 += check_box(board, 0, 1, "left", 1)
# print_board_ascii(board, 7, 1)

# print(board[0,0])  # Should be [1,1,1,1] => all edges drawn
# print("player 1 has", completed_boxes_player1, "boxes") # Should be 2
# print("player 2 has", completed_boxes_player2, "boxes") # Should be 0


In [19]:
def gameplay():
# Initializing board and vars
  n = int(input("Enter board size to start: "))
  game = dots_and_boxes(n)
  move = 1
  player = 1
  game.print_board_ascii(move, player, initial = True)

# Players take turns until all boxes made
  while (len(game.boxes_playerA) + len(game.boxes_playerB)) < n**2:
    if(game.On_Offensive):
      print(f"Player {player}'s turn again!\n")
    else:
      print(f"Player {player}'s turn!")


    # Getting parameters for edge to draw
    s = input("Enter the box to place edge as (row,column): ")
    s = s.strip()
    s = s.strip("(").strip(")")
    r,c = s.split(",")
    r = int(r)
    c = int(c)
    edge_type = input("Select what edge to place the line on from 'top', 'bottom', 'left', and 'right': ")
    print()

    # Making changes to board based on edge drawn
    _ = game.player_turn(r, c, edge_type, player, move)

    # Updating parameters for next turn
    move += 1
    if not game.On_Offensive:
      if player == 1:
        player = 2
      else:
        player = 1

  # Checking who won afer game ended
  if (len(game.boxes_playerA) > len(game.boxes_playerB)):
    print("Congratulations! Player 1 wins!")
  elif(len(game.boxes_playerB) > len(game.boxes_playerA)):
    print("Congratulations! Player 2 wins!")
  else:
    print("Woah! It's a tie!")


In [20]:
gameplay()

Enter board size to start: 2

      Welcome!     
 Here is the board 
 +       +       +
   (0,0)   (0,1)  
 +       +       +
   (1,0)   (1,1)  
 +       +       +

Player 1's turn!
Enter the box to place edge as (row,column): (0,0)
Select what edge to place the line on from 'top', 'bottom', 'left', and 'right': bottom

 Player 1 , Move 1 
 +       +       +
   (0,0)   (0,1)  
 +-------+       +
   (1,0)   (1,1)  
 +       +       +

Player 2's turn!
Enter the box to place edge as (row,column): (0,1)
Select what edge to place the line on from 'top', 'bottom', 'left', and 'right': left

 Player 2 , Move 2 
 +       +       +
   (0,0) | (0,1)  
 +-------+       +
   (1,0)   (1,1)  
 +       +       +

Player 1's turn!
Enter the box to place edge as (row,column): (0,0)
Select what edge to place the line on from 'top', 'bottom', 'left', and 'right': top

 Player 1 , Move 3 
 +-------+       +
   (0,0) | (0,1)  
 +-------+       +
   (1,0)   (1,1)  
 +       +       +

Player 2's turn!
Ent